# Ch6 - 資料載入、儲存和檔案格式

In [1]:
import pandas as pd
import numpy as np

## pandas中的解析函式
|函式|描述|
|:-:|:-:|
|read_csv|從檔案、URL或類檔案物件中讀取符號分隔資料，預設使用逗號分隔|
|read_table|從檔案、URL或類檔案物件中讀取符號分隔資料，預設使用Tab('\t')分隔|
|read_fwf|從固定寬度欄位讀取資料(例如:沒有分隔符號)|
|read_clipboard|類似read_table，用來從剪貼簿讀取資料，在轉換網頁資料為表格時很好用|
|read_excel|從 Excel XLS 或 XLSX 讀取表格式資料|
|read_hdf|讀取用pandas寫出的HDF5檔案|
|read_html|從指定的HTML文件中讀取所有的表格|
|read_json|從JSON(JavaScript Object notation)字串格式讀取資料|
|read_msgpack|讀取以MessagePack二進位格式編碼過的pandas資料|
|read_pickle|讀取以Python pickle格式所儲存的資料|
|read_sas|從SAS系統的儲存格式中讀取一個SAS資料集|
|read_sql|把一個SQL query(使用SQLAlchemy)的結果，讀成一個pandas DataFame|
|read_stata|從Stata檔案格式讀取資料集|
|read_feather|讀取Feather二進位格式|

* 上述函式都是將文字資料轉換為DataFrame，這些函式的可選參數，可大致分為以下幾種:
    * 索引(indexing):
        * 可以擇一或多個欄位作為DataFrame回傳，還有要不要從檔案中取得欄名稱，使用者名稱或是都不取得
<br><br>
    * 型態推斷與資料轉換:
        * 包括使用者定義值轉換，以其缺失值要用什麼取代
<br><br>
    * 解析日期時間格式:
        * 具組合功能，包括將跨多個欄位的日期和時間資料整合到單一欄中
<br><br>
    * 疊代(iterating):
        * 處理非常大的檔案時，支援多個區塊的疊代運作
<br><br>
    * 未清理的資料問題:
        * 跳過列或是註腳，註解或其他的小東西，例如數值格式資料標示一千的逗號

e.g. CSV文字檔

In [16]:
df = pd.read_csv(r'D:\Python\Python 資料分析\ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


* 使用read_table

In [18]:
pd.read_table(r'D:\Python\Python 資料分析\ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


* 讓pandas預設欄名稱

In [20]:
pd.read_csv(r'D:\Python\Python 資料分析\ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


* 自行指定欄名稱(使用names=[ ]參數)

In [23]:
pd.read_csv(r'D:\Python\Python 資料分析\ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


* 假設想讓message變成回傳DataFrame的index的話，可以藉由index_col參數指定index為4(message的index)或指定名稱'message'

In [24]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv(r'D:\Python\Python 資料分析\ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


* 假設想依多個欄的值做出階層式index -> 將要用的欄編號或名稱以list傳入

In [25]:
parsed = pd.read_csv(r'D:\Python\Python 資料分析\csv_mindex.csv', index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

* 某些情況下，資料表可能沒有固定的分隔符號，只用空白或其他東西做分隔

In [30]:
with open(r'D:\Python\Python 資料分析\ex3.txt') as f:
    print(list(f))

['            A         B         C\n', 'aaa -0.264438 -1.026059 -0.619500\n', 'bbb  0.927272  0.302904 -0.032399\n', 'ccc -0.264273 -0.386314 -0.217601\n', 'ddd -0.871858 -0.348382  1.100491']


* \s表示匹配任何空白字符，包括空格、制表符、换頁符等等,等價於[ \f\n\r\t\v]
* +在正则表达式中表示“匹配一次或多次”
* \s+则表示匹配任意多个上面的字符

In [31]:
result = pd.read_table(r'D:\Python\Python 資料分析\ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


* 上述範例中，因為欄位名稱那列的數量比資料列少一個，因此read_table推斷第一欄為DataFrame的索引

* 解析函式提供許多參數，可以處理可能會碰到的各種檔案格式的異常

In [41]:
with open(r'D:\Python\Python 資料分析\ex4.csv') as f:
    print(list(f))

['# 我要被跳過了,,,,\n', 'a,b,c,d,message\n', '# 包子,,,,\n', '# 一堆包子,,,,\n', '1,2,3,4,hello\n', '5,6,7,8,world\n', '9,10,11,12,foo\n']


* skirows可以用來跳過指定列

In [42]:
pd.read_csv(r'D:\Python\Python 資料分析\ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


* pandas會預設使用常用的標記值(NA或NULL)來當作缺失值的標記

In [10]:
result = pd.read_csv(r'D:\Python\Python 資料分析\ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [11]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


* na_values參數設定為一組字串，資料值符合字串者會被當作缺失值

In [12]:
result = pd.read_csv(r'D:\Python\Python 資料分析\ex5.csv', na_values=['world'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,NaN
2,three,9,10,11.0,12,foo


* 使用dict格式可為每個欄指定不同的NA標記

In [13]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv(r'D:\Python\Python 資料分析\ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


## 部分read_csv/read_table函式參數
|參數|描述|
|-:|:-:|
|path|表示檔案系統，URL或類檔案物件的字串|
|sep 或 delimiter|用來分隔每列欄位的字元序列或正規表達式|
|header|用來當作欄名的列號，預設為0(第一列)，如果沒有欄名列的話，應該要指定為None|
|index_col|指定用來當作列index的欄編號或名稱，可以指定單一欄或是為階層式index指定多個欄|
|names|和header=None合併使用，以list格式指定欄位名稱|
|skiprows|從檔案開始起算，要跳過的列號(首列為0)|
|na_values|要用NA取代掉的值|
|comment|見到指定字元，即認為式註解(一直到行尾)|
|parse_dates|嘗試將資料解析為datetime，預設為False。如果設定為True的話，會試圖把所有欄都解析為datatime，不然可以指定要解析的欄編號或名稱。如果傳遞的是list或tuple格式的話，將會合併多個欄的資料並解析成日期(例如: 日期/時間分別放在兩欄的情況)|
|keep_date_col|如果合併欄位進行日期解析的話，保留原始欄位，預設為False|
|converters|一個含欄編號或名稱，對應到函式的dict型態(例如: {'foo': f})，指定套用f函式於'foo'欄中的所有值|
|dayfirsts|當解析不好分辨日期時，套用國際格式(例如: 7/6/2012 -> Jun 7, 2012)，預設為False|
|date_parser|用來解析日期的格式|
|nrows|從檔頭開始算，要讀幾行|
|iterator|為分段讀取檔案，回傳一個TextParser物件|
|chunksize|疊代時使用，指定每段大小|
|skip_footer|跳過檔案的最後幾行|
|verbose|印出詳細的解析輸出訊息，例如在非數值欄為中有多少缺失值|
|encoding|Unicode的文字編碼(例如: 'utf-8'代表UTF-8編碼文字)|
|squeeze|如果解析的資料只有一個欄位，就回傳一個Series|
|thousands|千分位分隔符號(例如: ',' 或 '.')|